In [1]:
import json
import cv2
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model('emotion_detection_model.h5')

# Load the haarcascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define the emotion labels
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# Initialize emotion counts
emotion_counts = {label: 0 for label in emotion_labels.values()}
total_frames = 0
start_time = cv2.getTickCount() / cv2.getTickFrequency()

# Start capturing video from webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Failed to open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame from webcam.")
        break

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Detect faces in the frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # For each face detected, predict emotion
    for (x, y, w, h) in faces:
        face_roi = gray[y:y + h, x:x + w]
        face_roi = cv2.resize(face_roi, (48, 48))
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = np.expand_dims(face_roi, axis=-1)

        # Predict emotion
        predicted_emotions = model.predict(face_roi)
        emotion_label_arg = np.argmax(predicted_emotions)
        emotion_text = emotion_labels[emotion_label_arg]

        # Update emotion counts
        emotion_counts[emotion_text] += 1
        total_frames += 1

    # Check if 30 seconds have elapsed
    current_time = cv2.getTickCount() / cv2.getTickFrequency()
    elapsed_time = current_time - start_time
    if elapsed_time >= 30:
        break

    # Check if 'q' is pressed to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Calculate emotion percentages
emotion_percentages = {label: (count / total_frames) * 100 for label, count in emotion_counts.items()}

# Print emotion percentages
for label, percentage in emotion_percentages.items():
    print(f"{label}: {percentage:.2f}%")

# Save emotion percentages to a JSON file
with open('emotion_percentages.json', 'w') as f:
    json.dump(emotion_percentages, f)

# Release video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 32ms/step
Angry: 5.42%
Disgust: 0.00%
Fear: 2.46%
Happy: 11.82%
Sad: 31.03%
Surprise: 0.99%
Neutral: 48.28%
